In [1]:
!pip install -q langchain-community faiss-cpu tiktoken python-dotenv
!pip install -q langchain langchain_huggingface langchain_core
!pip install -q langchain-text-splitters
!pip install -q youtube-transcript-api
!pip install -q ragas  # For evaluation
!pip install -q rank-bm25  # For BM25 keyword search
!pip install -q sentence-transformers  # For reranking

In [2]:
import os
from typing import List, Dict, Any, Tuple
import numpy as np

# YouTube & Text Processing
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api._errors import TranscriptsDisabled

# LangChain Core
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import (
    RunnablePassthrough, 
    RunnableLambda, 
    RunnableParallel,
    RunnableBranch
)
from langchain_core.documents import Document

# LangChain - Hugging Face
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFaceEndpoint, ChatHuggingFace

# BM25 for Hybrid Search
from rank_bm25 import BM25Okapi

# Sentence Transformers for Reranking
from sentence_transformers import CrossEncoder

In [3]:
class Config:
    """Centralized configuration"""
    # API Keys
    HF_TOKEN = "your_token_here"

    
    # Video
    VIDEO_ID = "kCc8FmEb1nY"
    
    # Chunking
    CHUNK_SIZE = 1000
    CHUNK_OVERLAP = 200
    
    # Retrieval
    RETRIEVAL_K = 10
    FINAL_K = 4
    ALPHA = 0.5  # Weight for hybrid search (0.5 = equal weight)
    
    # Models
    EMBEDDING_MODEL = "BAAI/bge-base-en-v1.5"
    LLM_MODEL = "mistralai/Mistral-7B-Instruct-v0.2"
    RERANKER_MODEL = "cross-encoder/ms-marco-MiniLM-L-6-v2"
    
    # Memory
    MAX_HISTORY = 5

os.environ["HUGGING_FACE_ACCESS_TOKEN"] = Config.HF_TOKEN

In [4]:
class ConversationMemory:
    """Simple conversation memory with history management"""
    
    def __init__(self, max_history: int = Config.MAX_HISTORY):
        self.max_history = max_history
        self.history = []
    
    def add_interaction(self, question: str, answer: str):
        """Add Q&A pair to memory"""
        self.history.append({"question": question, "answer": answer})
        if len(self.history) > self.max_history:
            self.history = self.history[-self.max_history:]
    
    def get_context(self) -> str:
        """Format history for prompt"""
        if not self.history:
            return "No previous conversation."
        
        context = "Previous conversation:\n"
        for i, interaction in enumerate(self.history, 1):
            context += f"Q{i}: {interaction['question']}\n"
            context += f"A{i}: {interaction['answer']}\n\n"
        return context
    
    def clear(self):
        """Clear history"""
        self.history = []
    
    def __len__(self):
        return len(self.history)

In [5]:
def fetch_youtube_transcript(video_id: str) -> Tuple[str, dict]:
    """Fetch transcript from YouTube video"""
    try:
        fetched_transcript = YouTubeTranscriptApi().fetch(video_id, languages=['en'])
        transcript_obj = fetched_transcript.to_raw_data()
        transcript = " ".join([entry["text"] for entry in transcript_obj])
        
        metadata = {
            "video_id": video_id,
            "url": f"https://www.youtube.com/watch?v={video_id}",
            "length": len(transcript),
            "num_segments": len(transcript_obj)
        }
        
        return transcript, metadata
    
    except TranscriptsDisabled:
        raise Exception("⚠️ No captions available for this video")
    except Exception as e:
        raise Exception(f"⚠️ Error fetching transcript: {e}")


def chunk_transcript(transcript: str, metadata: dict) -> List[Document]:
    """Split transcript into chunks with metadata"""
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=Config.CHUNK_SIZE,
        chunk_overlap=Config.CHUNK_OVERLAP,
        separators=["\n\n", "\n", ". ", " ", ""]
    )
    
    chunks = splitter.create_documents(
        texts=[transcript],
        metadatas=[metadata]
    )
    
    # Add chunk IDs
    for i, chunk in enumerate(chunks):
        chunk.metadata["chunk_id"] = i
    
    return chunks

In [6]:
class HybridRetriever:
    """Hybrid retrieval combining FAISS and BM25 with reranking"""
    
    def __init__(
        self, 
        vector_store: FAISS,
        bm25_index: BM25Okapi,
        chunks: List[Document],
        reranker: CrossEncoder,
        k: int = Config.RETRIEVAL_K,
        final_k: int = Config.FINAL_K,
        alpha: float = Config.ALPHA
    ):
        self.vector_store = vector_store
        self.bm25_index = bm25_index
        self.chunks = chunks
        self.reranker = reranker
        self.k = k
        self.final_k = final_k
        self.alpha = alpha
    
    def retrieve(self, query: str) -> List[Document]:
        """Execute hybrid retrieval with reranking"""
        
        # 1. Semantic Search (FAISS)
        semantic_results = self.vector_store.similarity_search_with_score(query, k=self.k)
        
        # 2. Keyword Search (BM25)
        tokenized_query = query.lower().split()
        bm25_scores = self.bm25_index.get_scores(tokenized_query)
        bm25_top_indices = np.argsort(bm25_scores)[-self.k:][::-1]
        
        # 3. Combine Results
        combined_scores = {}
        
        # Add semantic results
        semantic_scores = [score for _, score in semantic_results]
        max_semantic = max(semantic_scores) if semantic_scores else 1
        
        for doc, score in semantic_results:
            doc_id = id(doc)
            normalized_score = score / max_semantic
            combined_scores[doc_id] = {
                'doc': doc,
                'score': self.alpha * normalized_score
            }
        
        # Add BM25 results
        max_bm25 = max(bm25_scores) if max(bm25_scores) > 0 else 1
        
        for idx in bm25_top_indices:
            doc = self.chunks[idx]
            doc_id = id(doc)
            normalized_score = bm25_scores[idx] / max_bm25
            
            if doc_id in combined_scores:
                combined_scores[doc_id]['score'] += (1 - self.alpha) * normalized_score
            else:
                combined_scores[doc_id] = {
                    'doc': doc,
                    'score': (1 - self.alpha) * normalized_score
                }
        
        # Sort by combined score
        sorted_results = sorted(
            combined_scores.values(),
            key=lambda x: x['score'],
            reverse=True
        )[:self.k]
        
        # 4. Reranking
        candidates = [item['doc'] for item in sorted_results]
        pairs = [[query, doc.page_content] for doc in candidates]
        rerank_scores = self.reranker.predict(pairs)
        
        # Get top final_k after reranking
        reranked_indices = np.argsort(rerank_scores)[-self.final_k:][::-1]
        final_docs = [candidates[i] for i in reranked_indices]
        
        return final_docs


class MultiQueryRetriever:
    """Multi-query retrieval for better coverage"""
    
    def __init__(
        self,
        base_retriever: HybridRetriever,
        llm: Any,
        num_queries: int = 3
    ):
        self.base_retriever = base_retriever
        self.llm = llm
        self.num_queries = num_queries
        
        # Multi-query prompt
        self.multi_query_prompt = ChatPromptTemplate.from_template("""
You are an AI assistant that generates multiple search queries.

Given a user question, generate 3 different versions of the question to retrieve relevant documents.
Provide these alternative questions separated by newlines.

Original question: {question}

Alternative questions:
""")
    
    def generate_queries(self, question: str) -> List[str]:
        """Generate multiple query variations"""
        try:
            prompt = self.multi_query_prompt.invoke({"question": question})
            response = self.llm.invoke(prompt)
            queries = [q.strip() for q in response.content.split('\n') if q.strip()]
            return [question] + queries[:self.num_queries-1]
        except:
            return [question]
    
    def retrieve(self, question: str) -> Tuple[List[Document], List[str]]:
        """Retrieve using multiple query variations and return both docs and queries"""
        queries = self.generate_queries(question)
        
        # Retrieve for each query
        all_docs = []
        seen_content = set()
        
        for query in queries:
            docs = self.base_retriever.retrieve(query)
            
            # Deduplicate
            for doc in docs:
                content_hash = hash(doc.page_content)
                if content_hash not in seen_content:
                    seen_content.add(content_hash)
                    all_docs.append(doc)
        
        # Final reranking if we have too many
        if len(all_docs) > Config.FINAL_K:
            pairs = [[question, doc.page_content] for doc in all_docs]
            rerank_scores = self.base_retriever.reranker.predict(pairs)
            reranked_indices = np.argsort(rerank_scores)[-Config.FINAL_K:][::-1]
            final_docs = [all_docs[i] for i in reranked_indices]
        else:
            final_docs = all_docs
        
        return final_docs, queries  # Return both docs and queries


In [7]:
def format_docs(docs: List[Document]) -> str:
    """Format documents for context"""
    return "\n\n---\n\n".join([doc.page_content for doc in docs])


def create_rag_chain(
    retriever: MultiQueryRetriever,
    llm: Any,
    memory: ConversationMemory
):
    """
    Create the main RAG chain with all components
    
    Chain Flow:
    1. Input: question
    2. Parallel: retrieve contexts + get memory
    3. Format context
    4. Build prompt with context + memory + question
    5. Generate answer
    6. Store in memory
    7. Return answer + contexts + queries
    """
    
    # RAG Prompt Template
    rag_prompt = ChatPromptTemplate.from_template("""
You are a helpful AI assistant answering questions about a YouTube video.

{conversation_history}

Use the following context from the video to answer the question.
If the context doesn't contain the answer, say you don't know.
Be concise but informative.

Context from video:
{context}

Current question: {question}

Answer:
""")
    
    # Define retrieval runnable that returns both docs and queries
    def retrieve_with_queries(q):
        docs, queries = retriever.retrieve(q)
        return {"docs": docs, "queries": queries}
    
    retrieval_runnable = RunnableLambda(retrieve_with_queries)
    
    # Define memory runnable
    memory_runnable = RunnableLambda(lambda _: memory.get_context())
    
    # Main chain
    chain = (
        # Step 1: Parallel retrieval and memory
        RunnableParallel({
            "question": RunnablePassthrough(),
            "retrieval_result": retrieval_runnable,
            "conversation_history": memory_runnable
        })
        # Step 2: Extract and format contexts
        .assign(
            contexts=lambda x: x["retrieval_result"]["docs"],
            queries=lambda x: x["retrieval_result"]["queries"],
            context=lambda x: format_docs(x["retrieval_result"]["docs"])
        )
        # Step 3: Generate answer
        .assign(
            answer=rag_prompt | llm | StrOutputParser()
        )
    )
    
    return chain


def create_chat_function(chain: Any, memory: ConversationMemory):
    """
    Wrap chain with memory storage and return answer, contexts, and queries
    """
    
    def chat(question: str, verbose: bool = True) -> Tuple[str, List[Document], List[str]]:
        """
        Chat function that handles memory and returns answer + contexts + queries
        
        Args:
            question: User's question
            verbose: Print processing info
            
        Returns:
            (answer, retrieved_contexts, generated_queries)
        """
        if verbose:
            print(f"\n{'='*80}")
            print(f"❓ Question: {question}")
            print(f"{'='*80}\n")
        
        try:
            # Run chain
            result = chain.invoke(question)
            
            answer = result["answer"].strip()
            contexts = result["contexts"]
            queries = result["queries"]
            
            # Store in memory
            memory.add_interaction(question, answer)
            
            if verbose:
                print(f"🔍 Generated Queries:")
                for i, q in enumerate(queries, 1):
                    print(f"   {i}. {q}")
                print(f"\n🤖 Answer: {answer}")
                print(f"\n💡 (Retrieved {len(contexts)} relevant chunks)")
                print(f"{'='*80}\n")
            
            return answer, contexts, queries
        
        except Exception as e:
            print(f"\n⚠️ Error: {e}\n")
            raise
    
    return chat

In [8]:
def create_interactive_chat(chat_function: Any, memory: ConversationMemory, video_url: str):
    """Create interactive chat loop"""
    
    def interactive_chat():
        """
        Interactive chatbot session
        
        Commands:
        - 'quit' or 'exit': End chat
        - 'clear': Clear conversation memory
        - 'history': Show conversation history
        """
        
        print("\n" + "="*80)
        print("💬 YOUTUBE VIDEO CHATBOT - Interactive Mode")
        print("="*80)
        print(f"\n🔹 Video: {video_url}")
        print("\n🎮 Commands:")
        print("   - Type your question to chat")
        print("   - 'quit' or 'exit': End chat")
        print("   - 'clear': Clear conversation history")
        print("   - 'history': Show conversation history")
        print("\n" + "="*80 + "\n")
        
        while True:
            user_input = input("You: ").strip()
            
            if user_input.lower() in ['quit', 'exit', 'q']:
                print("\n👋 Goodbye! Thanks for chatting!\n")
                break
            
            elif user_input.lower() == 'clear':
                memory.clear()
                print("🧹 Conversation history cleared!\n")
                continue
            
            elif user_input.lower() == 'history':
                print("\n📜 Conversation History:")
                print("="*60)
                if len(memory) == 0:
                    print("(No conversation history yet)")
                else:
                    print(memory.get_context())
                print("="*60 + "\n")
                continue
            
            elif not user_input:
                continue
            
            # Get answer
            try:
                answer, docs, queries = chat_function(user_input, verbose=False)
                
                print(f"\n🔍 Generated Queries:")
                for i, q in enumerate(queries, 1):
                    print(f"   {i}. {q}")
                
                print(f"\n🤖 Bot: {answer}")
                print(f"\n💡 (Retrieved {len(docs)} relevant chunks)\n")
            
            except Exception as e:
                print(f"\n⚠️ Error: {e}\n")
    
    return interactive_chat

In [10]:
def setup_chatbot(
    video_id: str = Config.VIDEO_ID,
    use_multi_query: bool = True
):
    """
    Main setup function - initializes everything and returns chat functions
    
    Args:
        video_id: YouTube video ID
        use_multi_query: Whether to use multi-query retrieval (slower but better)
        
    Returns:
        chat_function, interactive_chat_function, memory, metadata
    """
    
    print("🚀 Setting up YouTube Chatbot V2...\n")
    
    # Step 1: Fetch transcript
    print("📥 Fetching transcript...")
    transcript, metadata = fetch_youtube_transcript(video_id)
    print(f"✅ Fetched {metadata['length']:,} characters\n")
    
    # Step 2: Chunk transcript
    print("✂️ Chunking transcript...")
    chunks = chunk_transcript(transcript, metadata)
    print(f"✅ Created {len(chunks)} chunks\n")
    
    # Step 3: Create embeddings and vector store
    print("🧮 Creating embeddings and vector store...")
    embeddings = HuggingFaceEmbeddings(
        model_name=Config.EMBEDDING_MODEL,
        model_kwargs={'device': 'cpu'},
        encode_kwargs={'normalize_embeddings': True}
    )
    vector_store = FAISS.from_documents(chunks, embeddings)
    print(f"✅ Vector store created\n")
    
    # Step 4: Create BM25 index
    print("📚 Creating BM25 index...")
    tokenized_docs = [doc.page_content.lower().split() for doc in chunks]
    bm25_index = BM25Okapi(tokenized_docs)
    print(f"✅ BM25 index created\n")
    
    # Step 5: Load reranker
    print("🎯 Loading reranker...")
    reranker = CrossEncoder(Config.RERANKER_MODEL)
    print(f"✅ Reranker loaded\n")
    
    # Step 6: Initialize LLM
    print("🤖 Initializing LLM...")
    llm = HuggingFaceEndpoint(
        repo_id=Config.LLM_MODEL,
        huggingfacehub_api_token=Config.HF_TOKEN,
        max_new_tokens=512,
        temperature=0.7,
        top_p=0.95
    )
    chat_model = ChatHuggingFace(llm=llm)
    print(f"✅ LLM initialized\n")
    
    # Step 7: Create retrievers
    print("🔍 Creating retrieval system...")
    hybrid_retriever = HybridRetriever(
        vector_store=vector_store,
        bm25_index=bm25_index,
        chunks=chunks,
        reranker=reranker
    )
    
    if use_multi_query:
        retriever = MultiQueryRetriever(
            base_retriever=hybrid_retriever,
            llm=chat_model
        )
        print(f"✅ Multi-query retrieval enabled\n")
    else:
        retriever = hybrid_retriever
        print(f"✅ Standard hybrid retrieval enabled\n")
    
    # Step 8: Initialize memory
    print("💭 Initializing conversation memory...")
    memory = ConversationMemory()
    print(f"✅ Memory initialized\n")
    
    # Step 9: Create chain
    print("⛓️ Building RAG chain...")
    chain = create_rag_chain(
        retriever=retriever,
        llm=chat_model,
        memory=memory
    )
    print(f"✅ Chain created\n")
    
    # Step 10: Create chat functions
    chat_function = create_chat_function(chain, memory)
    interactive_chat_function = create_interactive_chat(
        chat_function, 
        memory, 
        metadata['url']
    )
    
    print("="*80)
    print("✅ SETUP COMPLETE!")
    print("="*80)
    print("\n📊 System Summary:")
    print(f"   • Video: {metadata['url']}")
    print(f"   • Chunks: {len(chunks)}")
    print(f"   • Retrieval: {'Multi-Query' if use_multi_query else 'Hybrid'}")
    print(f"   • Reranking: Enabled")
    print(f"   • Memory: Last {Config.MAX_HISTORY} interactions")
    print("\n🎯 Ready to chat!\n")
    print("Usage:")
    print("  answer, contexts = chat_function('Your question here')")
    print("  interactive_chat_function()  # Start interactive chat")
    print("="*80 + "\n")
    
    return chat_function, interactive_chat_function, memory, metadata

In [11]:
if __name__ == "__main__":
    """
    Example usage of the chatbot
    """
    
    # Setup chatbot (now always uses multi-query)
    chat_function, interactive_chat, memory, metadata = setup_chatbot()
    
    # Example 1: Single question
    print("\n" + "🧪"*40)
    print("TESTING SINGLE QUESTION")
    print("🧪"*40 + "\n")
    
    answer, contexts, queries = chat_function("What is this video about?")
    
    # Example 2: Follow-up question (uses memory)
    answer, contexts, queries = chat_function("Can you explain it in more detail?")
    
    # Example 3: Another question
    answer, contexts, queries = chat_function("What are the main concepts?")
    
    # Example 4: Start interactive chat
    print("\n" + "💬"*40)
    print("STARTING INTERACTIVE CHAT")
    print("💬"*40 + "\n")
    
    # Uncomment to start interactive mode:
    # interactive_chat()

🚀 Setting up YouTube Chatbot V2...

📥 Fetching transcript...
✅ Fetched 108,990 characters

✂️ Chunking transcript...
✅ Created 144 chunks

🧮 Creating embeddings and vector store...
✅ Vector store created

📚 Creating BM25 index...
✅ BM25 index created

🎯 Loading reranker...
✅ Reranker loaded

🤖 Initializing LLM...
✅ LLM initialized

🔍 Creating retrieval system...
✅ Multi-query retrieval enabled

💭 Initializing conversation memory...
✅ Memory initialized

⛓️ Building RAG chain...
✅ Chain created

✅ SETUP COMPLETE!

📊 System Summary:
   • Video: https://www.youtube.com/watch?v=kCc8FmEb1nY
   • Chunks: 144
   • Retrieval: Multi-Query
   • Reranking: Enabled
   • Memory: Last 5 interactions

🎯 Ready to chat!

Usage:
  answer, contexts = chat_function('Your question here')
  interactive_chat_function()  # Start interactive chat


🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪
TESTING SINGLE QUESTION
🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪


❓ Question: What is this video about?

🔍 Generated Queri

In [12]:
interactive_chat()


💬 YOUTUBE VIDEO CHATBOT - Interactive Mode

🔹 Video: https://www.youtube.com/watch?v=kCc8FmEb1nY

🎮 Commands:
   - Type your question to chat
   - 'quit' or 'exit': End chat
   - 'clear': Clear conversation history
   - 'history': Show conversation history




You:  what is happening in this video?



🔍 Generated Queries:
   1. what is happening in this video?
   2. 1. Could you describe the events taking place in this video?
   3. 2. What actions are depicted in this video?

🤖 Bot: In this video, the speaker is implementing a decoder-only Transformer model for text generation using Python and Jupyter Notebook. They are focusing on the self-attention mechanism and cross-attention in the model. The encoder reads French text, and all tokens are allowed to communicate with each other during encoding. In the decoder, there is an additional connection to the encoder outputs through a cross attention, where keys and values come from the encoder outputs. The speaker also discusses the use of embedding tables and logits in predicting the next character based on token identity. They assume prior knowledge of calculus, statistics, and neural networks, particularly the concepts introduced in the speaker's previous videos on the same channel. The goal is to develop a language model using the T

You:  exit



👋 Goodbye! Thanks for chatting!

